In [118]:
import os, shutil
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras import models, layers, optimizers
from keras import preprocessing
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras import backend as K
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, SimpleRNN, LSTM

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

from utils import plot_val_acc_loss

## Deep learning for text and sequences

#### Embedding layer with Dense layer

In [2]:
from keras.datasets import imdb

In [3]:
max_features = 10000
maxlen = 20
(x_train, y_train), (x_test, y_test) = imdb.load_data(
    num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [4]:
model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 8)             80000     
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [5]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 3s 3ms/step - loss: 0.6740 - acc: 0.6082 - val_loss: 0.6313 - val_acc: 0.6984
Epoch 2/10
625/625 [==============================] - 2s 2ms/step - loss: 0.5555 - acc: 0.7513 - val_loss: 0.5345 - val_acc: 0.7264
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4675 - acc: 0.7855 - val_loss: 0.5038 - val_acc: 0.7414
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4225 - acc: 0.8104 - val_loss: 0.4955 - val_acc: 0.7526
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3918 - acc: 0.8262 - val_loss: 0.4957 - val_acc: 0.7538
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3658 - acc: 0.8406 - val_loss: 0.4985 - val_acc: 0.7538
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3433 - acc: 0.8528 - val_loss: 0.5034 - val_acc: 0.7510
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0.

In [17]:
plot_val_acc_loss(history)

#### Pretrained embeddings

In [52]:
def extract_reviews(path):
    labels = []
    texts = []
    for label_type in ['neg', 'pos']:
        dir_name = os.path.join(path, label_type)
        for fname in os.listdir(dir_name):
            if fname[-4:] == '.txt':
                f = open(os.path.join(dir_name, fname), encoding = 'cp850')
                texts.append(f.read())
                f.close()
                if label_type == 'neg':
                    labels.append(0)
                else:
                    labels.append(1)
    return texts, labels

In [53]:
imdb_dir = r'C:\Users\Cyril\Downloads\aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

texts, labels = extract_reviews(train_dir)

In [13]:
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [25]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

Found 88583 unique tokens.


In [26]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [33]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

In [35]:
glove_dir = r'C:\Users\Cyril\Downloads\glove.6B'

embedding_index = {}
with open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding = 'cp850') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs
print(f'Found {len(embedding_index)} word vectors')

Found 400000 word vectors


In [38]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [43]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 32)                320032    
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [47]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
7/7 [==============================] - 12s 2s/step - loss: 1.5742 - acc: 0.4750 - val_loss: 0.9562 - val_acc: 0.4985
Epoch 2/10
7/7 [==============================] - 1s 145ms/step - loss: 0.5976 - acc: 0.6950 - val_loss: 0.6893 - val_acc: 0.5501
Epoch 3/10
7/7 [==============================] - 3s 546ms/step - loss: 0.4360 - acc: 0.7850 - val_loss: 0.7092 - val_acc: 0.5488
Epoch 4/10
7/7 [==============================] - 7s 1s/step - loss: 0.3268 - acc: 0.8650 - val_loss: 0.7175 - val_acc: 0.5517
Epoch 5/10
7/7 [==============================] - 8s 1s/step - loss: 0.3013 - acc: 0.8750 - val_loss: 0.6915 - val_acc: 0.5571
Epoch 6/10
7/7 [==============================] - 1s 174ms/step - loss: 0.1203 - acc: 1.0000 - val_loss: 0.7943 - val_acc: 0.5456
Epoch 7/10
7/7 [==============================] - 1s 170ms/step - loss: 0.1618 - acc: 0.9550 - val_loss: 1.0983 - val_acc: 0.5213
Epoch 8/10
7/7 [==============================] - 4s 655ms/step - loss: 0.0573 - acc: 1.0000 - val

In [48]:
plot_val_acc_loss(history)

In [56]:
test_dir = os.path.join(imdb_dir, 'test')
texts, labels = extract_reviews(test_dir)

In [57]:
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [58]:
model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)

782/782 [==============================] - 9s 12ms/step - loss: 0.7803 - acc: 0.5629


[0.7802525758743286, 0.5628799796104431]

#### RNNs and LSTMs

In [61]:
from keras.datasets import imdb
max_features = 10000
maxlen = 500
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(
    num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('input shape:', x_train.shape)
print('test shape:', x_test.shape)

Loading data...
input shape: (25000, 500)
test shape: (25000, 500)


In [63]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 28s 169ms/step - loss: 0.6435 - acc: 0.6074 - val_loss: 0.4719 - val_acc: 0.7924
Epoch 2/10
157/157 [==============================] - 26s 164ms/step - loss: 0.4056 - acc: 0.8298 - val_loss: 0.4237 - val_acc: 0.8102
Epoch 3/10
157/157 [==============================] - 23s 150ms/step - loss: 0.2887 - acc: 0.8848 - val_loss: 0.3440 - val_acc: 0.8592
Epoch 4/10
157/157 [==============================] - 25s 158ms/step - loss: 0.2133 - acc: 0.9197 - val_loss: 0.3891 - val_acc: 0.8448
Epoch 5/10
157/157 [==============================] - 23s 148ms/step - loss: 0.1535 - acc: 0.9435 - val_loss: 0.4047 - val_acc: 0.8454
Epoch 6/10
157/157 [==============================] - 27s 171ms/step - loss: 0.1183 - acc: 0.9603 - val_loss: 0.6377 - val_acc: 0.7348
Epoch 7/10
157/157 [==============================] - 24s 153ms/step - loss: 0.0672 - acc: 0.9794 - val_loss: 0.5565 - val_acc: 0.8086
Epoch 8/10
157/157 [==============================] - 2

In [116]:
plot_val_acc_loss(history)

In [119]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

In [120]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 52s 310ms/step - loss: 0.5279 - acc: 0.7537 - val_loss: 0.3620 - val_acc: 0.8570
Epoch 2/10
157/157 [==============================] - 47s 297ms/step - loss: 0.3063 - acc: 0.8778 - val_loss: 0.4445 - val_acc: 0.8346
Epoch 3/10
157/157 [==============================] - 47s 299ms/step - loss: 0.2485 - acc: 0.9049 - val_loss: 0.2899 - val_acc: 0.8798
Epoch 4/10
157/157 [==============================] - 50s 316ms/step - loss: 0.2063 - acc: 0.9237 - val_loss: 0.2853 - val_acc: 0.8876
Epoch 5/10
157/157 [==============================] - 51s 327ms/step - loss: 0.1803 - acc: 0.9351 - val_loss: 0.3178 - val_acc: 0.8654
Epoch 6/10
157/157 [==============================] - 56s 351ms/step - loss: 0.1621 - acc: 0.9424 - val_loss: 0.4272 - val_acc: 0.8714
Epoch 7/10
157/157 [==============================] - 51s 324ms/step - loss: 0.1459 - acc: 0.9482 - val_loss: 0.3117 - val_acc: 0.8808
Epoch 8/10
157/157 [==============================] - 5

In [121]:
plot_val_acc_loss(history)